In [3]:
pip install praw



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os


In [62]:
import praw
from prawcore.exceptions import NotFound
import pandas as pd
from tqdm import tqdm

arr=[]
reddit = praw.Reddit(
    client_id='eLHIfVLEUxnTQlLlGt29_Q',
    client_secret='xalSQxSsLr5oxApVsrobL5dyasmCOw',
    user_agent='python:my_reddit_sentiment_app:v1.0.0 (by u/on_a_no_buy)'
)

try:
    subreddit = reddit.subreddit('sneakers')
    for post in tqdm(subreddit.hot(limit=1000)):
        #print(post.title)
        #print(post.selftext)
        if len(post.comments) > 0:
            comment=post.comments[0].body
        else:
            comment=" "
        arr.append([post.title,post.selftext,post.upvote_ratio,post.created_utc,post.score,comment])
except NotFound:
    print("The requested resource was not found. Please check the subreddit name or resource ID.")
df_sneakers = pd.DataFrame(arr, columns=['Title', 'Body','Upvote_ratio','Timestamp','Upvote_num','comments'])


990it [07:35,  2.17it/s]


In [63]:
df_sneakers

,Title,Body,Upvote_ratio,Timestamp,Upvote_num,comments
0,Nikon releases the 125x optical zoom COOLPIX P...,,0.97,1.738758e+09,73,"Other than the USB-C port update, I’m not real..."
1,Z 35mm f/1.2 S available for preorder,,1.00,1.738728e+09,65,"Man, that took forever. I remember fall of 20..."
2,Is this the last king of the film era?,,0.98,1.738936e+09,201,"Yeppers, only reason why I haven’t picked up o..."
3,Welcoming the newest member of the family,,0.95,1.738951e+09,85,Summoning their ultimate successor...
4,Z6ii with 70-300,,0.98,1.738958e+09,32,"The Tamron 70-300? Great lens, I want one."
...,...,...,...,...,...,...
985,NY Wildlife (zfc + 50 1.8s),,0.99,1.737651e+09,37,Exceptional shots! How did you get so close wi...
986,D850 RAW converter?,Opinions on the best RAW converter for the D85...,1.00,1.737688e+09,5,Best is up to you. There's literally hundreds ...
987,What are the best cameras for a beginner?,**Feeling Overwhelmed by the Camera Market?**\...,0.56,1.737719e+09,1,This gets asked roughly every 15 minutes in r/...
988,Walking back from a fun event at Ventura harbo...,Nikon D500 and 35mm 1.8 with a diffusion filt...,0.75,1.737703e+09,2,


In [64]:
df_sneakers.to_csv('sneakers.csv', index=False)


In [65]:
# Path to the folder containing the CSV files
folder_path = 'Dataframe_new'

# List to store the DataFrames
dataframes = []

# Iterate over all files in the folder
for file_name in os.listdir(folder_path):
    # Check if the file is a CSV
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV into a DataFrame
        df = pd.read_csv(file_path)
        df['topic'] = os.path.splitext(file_name)[0]
        dataframes.append(df)
        print(f"Loaded: {file_name}")

print(f"Total CSV files read: {len(dataframes)}")

Loaded: google.csv
Loaded: teslamotors.csv
Loaded: playstation.csv
Loaded: nikon.csv
Loaded: apple.csv
Loaded: technology.csv
Loaded: xbox.csv
Loaded: amd.csv
Loaded: intel.csv
Loaded: tsmc.csv
Loaded: canon.csv
Loaded: dell.csv
Loaded: meta.csv
Loaded: microsoft.csv
Loaded: samsung.csv
Loaded: nvidia.csv
Total CSV files read: 16


In [66]:
combined_df = pd.concat(dataframes, ignore_index=True)


In [67]:
combined_df

,Title,Body,Upvote_ratio,Timestamp,Upvote_num,comments,topic
0,Support Megathread - November 2023,Have a question you need answered? A new Googl...,0.98,1.698797e+09,68,"What's the point of this ""Megathread"" if no on...",google
1,Oh yeah makes sense,NaN,0.92,1.738574e+09,181,This is why I blocked ai overview using uBlock...,google
2,Google says Gemini is being misused to launch ...,NaN,0.95,1.738586e+09,62,>Dozens of cybercriminal organizations from al...,google
3,Is this the reason for dark web alerts today? ...,NaN,0.94,1.738541e+09,376,If this isn't an official government job how i...,google
4,“Impeach Do” and “Impeach Tr” makes google hav...,NaN,0.86,1.738539e+09,319,DuckDuck though lol... 3rd letter in\n\nhttps:...,google
...,...,...,...,...,...,...,...
11651,With manufacturing nodes slowing down….the fut...,"We're approaching atomic limits with silicon, ...",0.84,1.737018e+09,93,>We're approaching atomic limits with silicon\...,nvidia
11652,"Wow, just tried DLDSR + DLSS on a 1440p screen.","With the launch of the 5000 series cards, I've...",0.94,1.736986e+09,433,"DLDSR is great, a shame that nvidia can't be b...",nvidia
11653,Doubts about the release of the 50 series,"Well, it's my first time buying online, I woul...",0.30,1.737153e+09,0,I'm having Vietnam flashbacks of the 30xx seri...,nvidia
11654,Alan Wake 2 will be the first game to support ...,NaN,0.96,1.736972e+09,723,Remedy has always been first to jump on new te...,nvidia


In [68]:
combined_df.to_csv('athletic_apparel.csv', index=False)


In [61]:
nan_per_column = combined_df.isnull().sum()
nan_per_column

Title       0
Body     2824
topic       0
dtype: int64